<a href="https://colab.research.google.com/github/mcondemaita/python_projects/blob/main/Reonomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reonomy is trying to determine the **owner** of a property using internal data about the property and external data about **business entities**.

In [2]:
import pandas  as pd #for the data frame
import numpy as np
import matplotlib.pyplot as plt
import requests

In [4]:
##Use pandas to read the information 
properties = pd.read_csv("/content/properties.csv")
#DataFrame has 0 - 274 rows and 202 columns 
properties.info()
#list(properties.columns.values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Columns: 202 entries, property_id to ownership
dtypes: bool(3), float64(101), int64(4), object(94)
memory usage: 428.5+ KB


Q1 

---
What are the major categories of data / columns? Which columns seem the most relevant for
determining the **owner of a property?** Prepare a few questions to discuss. (15 min)

In [5]:
#The major categories of data set that I woul select will be data1 
data1 = properties[['property_id','owner_std1_first_name','owner_std1_last_name', 'owner_std_name1_full','owner_std1_entity_type']]

#We may consider std2 but some data is missing 
data2 = properties[['property_id','owner_std2_first_name','owner_std2_last_name','owner_std_name2_full', 'owner_std2_entity_type']]

data1.isnull().sum()
#data2.isnull().sum()
# The following is missing: 
# owner_std2_first_name = 180 of 275 ~ 34% of data missing 
# owner_std2_last_name = 168 on a total data of 275 entries (rows) ~ 38% of data missing 


property_id                0
owner_std1_first_name     91
owner_std1_last_name       2
owner_std_name1_full       0
owner_std1_entity_type     0
dtype: int64

Q2. 

---
Create a new tab where you clean and edit the data to 
focus on owner-related fields. For
example, you might choose to hide or delete columns, rearrange columns, add background
colors, etc. (15 min)

In [32]:
drp_df = data1.drop(columns='property_id')
drp_df.head()

,owner_std1_first_name,owner_std1_last_name,owner_std_name1_full,owner_std1_entity_type
0,JOHN,GRUGEL,GRUGEL JOHN,person
1,ALAN,FISHMAN,FISHMAN ALAN L,person
2,JOSEPH,RICE,RICE JOSEPH L,person
3,LOKESH,KAPA,KAPA LOKESH,business entity
4,NaN,CITY OF GREENWOOD,CITY OF GREENWOOD,business entity


In [34]:
#Hightlight NaN Values in the DataFrame
#drp_df.style.applymap(nul_color = 'red')

In [96]:
#Rearrange columns 
drp_df.loc[:,['owner_std_name1_full','owner_std1_first_name','owner_std1_last_name','owner_std1_entity_type']]
index_ = drp_df.set_index('owner_std_name1_full').sort_values(by = 'owner_std_name1_full', ascending= True)
index_

,owner_std1_first_name,owner_std1_last_name,owner_std1_entity_type
owner_std_name1_full,,,
115 DOBBIN LLC,115 DOBBIN LLC,115 DOBBIN LLC,business entity
3255 TAMIAMI TRAIL LLC,NaN,3255 TAMIAMI TRAIL LLC,business entity
370 EAST 162 STREET LLC,NaN,370 EAST 162 STREET LLC,business entity
8000 DEL DONORE HOMEOWNERS ASSOCIATION INC,NaN,8000 DEL DONORE HOMEOWNERS ASSOCIATION INC,business entity
993 JOHNNIE DODDS LLC,NaN,993 JOHNNIE DODDS LLC,person
...,...,...,...
WILLIAMS MAXIE D,MAXIE,WILLIAMS,person
WILLIAMSON ROBERT JEROME,ROBERT,WILLIAMSON,person
WILLSON JAMES A,JAMES,WILLSON,person


In [ ]:
def color_groups(new_df):
  
  colors = ['red','black']
  x = index_.copy()
  names = list(x['owner_std1_first_name'].isnull())
  i = 0

  for name in names: 
    style = f'background-color: { colors[i]}'
    x.loc[x['owner_std1_first_name']== name, :] = style
    i = not i

  return x

color_df = new_df.reset_index().style.applymap(color_groups)
print(color_df.render())

Q3. 

---


What sort of information is available on the NY Department of State website? Prepare some
notes on the data provided, what you think it means, and how it might be useful. (15 min)

In the NYS web, we found information such as Entity names, County, Entity Type, name history that could help us unlock CRE insights to assit internal and external user with accurate and updated data. 

Q4. 

---


Pick out a few properties with owners that you think you can find on the NY site. Create a new tab with the property IDs and owner info for the selected properties. Search for the owners on the NY site, and record info about what you find, if anything. (30 min)

In [113]:
#To search for properties on the NY site we can use python to request and web scraping.
#The code below correlates with serchValue and our owner_std_name1_full. 

searchValue = 'Blue Rock Assets LLC'
searchUrl = "https://apps.dos.ny.gov/PublicInquiryWeb/api/PublicInquiry/GetComplexSearchMatchingEntities"
payload = {
    "searchValue": searchValue,
    "searchByTypeIndicator": "EntityName",
    "searchExpressionIndicator": "BeginsWith",
    "entityStatusIndicator": "AllStatuses",
    "entityTypeIndicator": [
        "Corporation",
        "LimitedLiabilityCompany",
        "LimitedPartnership",
        "LimitedLiabilityPartnership"
    ],
    "listPaginationInfo": {
        "listStartRecord": 1,
        "listEndRecord": 50
    }
}
req = requests.post(searchUrl, json=payload)
try: 
  data = req.json()
except IOError: 
  print("Unable to load data")

else: 
  print('The file load successfully ')
dosId = data["entitySearchResultList"][0]["dosID"]
print('DOS ID:', dosId)
print('Thus, this information corretate with our property DataFrame')

The file load successfully 
DOS ID: 4522164
Thus, this information corretate with our property DataFrame


In [72]:
#Properti 1= Blue Rock Assets
getEntityByIdUrl = "https://apps.dos.ny.gov/PublicInquiryWeb/api/PublicInquiry/GetEntityRecordByID"
entityPayload = {
      "SearchID": dosId,
      "AssumedNameFlag": "False "
  }

reqEntity = requests.post(getEntityByIdUrl, json=entityPayload)
print(reqEntity.json())

{'timestamp': '0001-01-01T00:00:00', 'requestStatus': 'Success', 'resultIndicator': 'EntityMatchFound', 'entityGeneralInfo': {'entityName': 'BLUE ROCK ASSETS LLC', 'dosID': '4522164', 'foreignLegalName': '', 'fictitiousName': '', 'entityType': 'DOMESTIC LIMITED LIABILITY COMPANY', 'durationDate': '', 'sectionofLaw': '203 LLC - LIMITED LIABILITY COMPANY LAW ', 'entityStatus': 'Active', 'dateOfInitialDosFiling': '2014-01-30T00:00:00', 'reasonForStatus': '', 'effectiveDateInitialFiling': '2014-01-30T00:00:00', 'inactiveDate': '', 'foreignFormationDate': '', 'statementStatus': 'PAST DUE DATE', 'county': 'Nassau', 'nextStatementDueDate': '2016-01-31T00:00:00', 'jurisdiction': 'New York, United States', 'nfpCategory': ''}, 'sopAddress': {'name': 'BLUE ROCK ASSETS LLC', 'address': {'streetAddress': 'PO BOX 231294', 'streetAddress1': 'PO BOX 231294', 'addressLine2': '', 'streetAddress2': '', 'city': 'GREAT NECK', 'state': 'NY', 'zipCode': '11021', 'zipPlus4': '', 'country': 'United States', 'c

Q5.

---
Any ideas on how we could find the owners of the property owners, e.g. the owner of "Blue
Rock Assets LLC"? You may want to poke around the NY website, or check out similar websites
for other states, or just take a few minutes to brainstorm. (15 min)

I would still use open resorches like the NY side as well as the DOB side. As a project manager assistant I submited inspection or permit on the DOB Now (Department of Builds) this side has more details on the owners and property information. We can also use web scraping here using python. 